# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [30]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, AciWebservice
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core import Workspace, Experiment, Dataset, Environment, ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.run import Run

import os
import requests
import json

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#Retrieve current workspace
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Tuscan_Wine_Forecast'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')



Workspace name: quick-starts-ws-162459
Azure region: southcentralus
Subscription id: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f
Resource group: aml-quickstarts-162459


## Setting up Environment

In [3]:
env = Environment(name="AzureML-Tutorial")

## Compute Cluster
If compute cluster already exists, it will just identify it and end.


In [4]:
#Create Cluster if does not already exist

cluster_name = "myCluster"
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Cluster already created")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_DS12_V2",min_nodes=1, max_nodes=6)
    cluster = ComputeTarget.create(ws,cluster_name, compute_config) #creates the actual cluster

cluster.wait_for_completion(show_output=True)  #Allows to continue on other threads while cluster is being created


#SOURCE / HELP: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute.amlcompute?view=azure-ml-py

InProgress.....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.......................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset
A Dataset is created from the csv file in the folder "data".

In [ ]:
# Why ??? => data picked from the train.py no need to create Dataset from the Notebook.

# Create a dataset from delimited files with header option as ONLY_FIRST_FILE_HAS_HEADERS


   #datastore = ws.get_default_datastore() 
   #dataset = Dataset.Tabular.from_delimited_files(path=(datastore, 'data/dataset_Tuscany_Wine_prepared.csv'),
   #    header='ONLY_FIRST_FILE_HAS_HEADERS')

  # df = dataset.to_pandas_dataframe()

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [13]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
 
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1, delay_evaluation=5)
#Ends runs when the primary metric isn't above (1/(1+0,1) ~90%. Start evaluating after 5 intervals.

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
        "--C": uniform(0.001, 0.5),
        "--max_iter": choice(5,10,25,100,10000)
})
#Random / Grid  https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters?view=azure-ml-py
#Bayesian sampling does not support early termination



#Logging metrics


run_logger = Run.get_context()
#run_logger.log("accuracy", float(val_accuracy))


#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(
    source_directory=os.path.join('./'),
    compute_target=cluster,
    entry_script="./Users/odl_user_162459/train.py"
)

#script_run_config = ScriptRunConfig(
    #source_directory=os.path.join('./'),
    #compute_target=cluster,
    #entry_script="./Users/odl_user_162459/train.py",
    #environment = env
#)


hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,
                                        policy = early_termination_policy,
                                        primary_metric_name="Accuracy",
                                        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                        max_duration_minutes = 30,
                                        max_concurrent_runs= 5,
                                        estimator= estimator,
                                        max_total_runs = 50
                                        #run_config=script_run_config
                                        )

In [14]:
#TODO: Submit your experiment

experiment = Experiment(ws, experiment_name)
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [15]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [17]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)



RunDetails(best_run).show()

Run(Experiment: Tuscan_Wine_Forecast,
Id: HD_41372462-f808-4ae1-96b4-f3fe00d9a4c8_17,
Type: azureml.scriptrun,
Status: Completed)


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [23]:
#TODO: Save the best model

os.makedirs('outputs',exist_ok=True)


# Register run as Model

model_name = "AleaumeModelHyperdrive"
description = "Best Hyperdrive Model"
model_path ="outputs/model.joblib"

#model = best_run.register_model(model_name = model_name, description = description, model_path = model_path )

model = best_run.register_model(model_name = model_name, description = description,model_path = model_path)

#SOURCE / HELP: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python

#Unfortunately so far unsuccessful to register the model via SDK and get a default generated swagger.json uri. Also,  MSFT support recommends to do so via AML studio...

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [28]:
#To create a local deployment configuration
deployment_config = LocalWebservice.deploy_configuration(port=9064)

#Create Environment

env = Environment(name="AzureML-AutoML")
#myenv=env.clone("myenv")
#myenv.python.conda_dependencies.add_pip_package("joblib==1.1.0")

#Uploaded manually entry scripts in the below defined "entry_script" path.

my_inference_config = InferenceConfig(
    environment=env,
    source_directory= './',
    entry_script="./Users/odl_user_162459/score.py"
    #entry_script="./score.py"
)



# Deploy the service locally

service = model.deploy(ws, "local-service", [model], my_inference_config, deployment_config)
service.reload()
print(service.get_logs())

print(service.scoring_uri)

service.wait_for_deployment(show_output=True)


Generating Docker build context.
Package creation Succeeded
Logging into Docker registry viennaglobal.azurecr.io
Logging into Docker registry viennaglobal.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM viennaglobal.azurecr.io/azureml/azureml_744753b4f832c74a6086091ff294e4a6
 ---> 7101fb6a62f0
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 263016436a31
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImEyNGEyNGQ1LThkODctNGM4YS05OWI2LTkxZWQyZDJkZjUxZiIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTE2MjQ1OSIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTE2MjQ1OSIsIndvcmtzcGFjZUlkIjoiNjg2ZDFiZmEtYTkyNC00NjAxLWJjYmUtYmY5ODRhODFlZmUyIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in b570063a7284
 ---> a71c2b90bb93
Step 4/5 : RUN mv '/var/azureml-app/tmpxsl5f6gd.py' /var/azureml-app/main.py
 ---> Running in 3f02724f87ea
 ---> 755113dd8476
Step 5/5 : 

TODO: In the cell below, send a request to the web service you deployed to test it.

In [31]:
#Call model to test 

#service.update(enable_app_insights=True)


uri = service.scoring_uri
requests.get("http://localhost:9064")
headers = {"Content-Type": "application/json"}
data = {
            "vintage":2016,
            "variety":"Sangiovese",
            "winery":"Casa Raia",
            "avg winter temp":4.2,
            "avg spring temp":15,
            "avg summer temp":27,
			"avg fall temp":16,
			"avg winter sun hour":7,
			"avg spring sun hour":12,
			"avg summer sun hour":16,
			"avg fall sun hour":9,
			"avg daily precip winter":3,
			"avg daily precip spring":3,
			"avg daily precip summer":1,
			"avg daily precip fall":2
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

{'data': "'data'", 'message': 'Failed to predict '}


In [32]:
#Deploy to ACI

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1, memory_gb=1, auth_enabled=True
)

service = model.deploy(
    ws,
    "mywebservice",
    [model],
    my_inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-10-31 15:08:51+00:00 Creating Container Registry if not exists.
2021-10-31 15:08:51+00:00 Registering the environment.
2021-10-31 15:08:52+00:00 Use the existing image.
2021-10-31 15:08:53+00:00 Submitting deployment to compute.
2021-10-31 15:08:57+00:00 Checking the status of deployment mywebservice..
2021-10-31 15:13:32+00:00 Checking the status of inference endpoint mywebservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
2021-10-31T15:13:18,188438500+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-10-31T15:13:18,188440200+00:00 - iot-server/run 
2021-10-31T15:13:18,211626300+00:00 - rsyslog/run 
2021-10-31T15:13:18,262893200+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_705720c76ff57b57c77d577152dabb18/lib/lib

In [33]:
import requests
import json

primary, secondary = service.get_keys()

# URL for the web service
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
key = primary

# Two sets of data to score, so we get two results back
data = {
			"vintage":2013,
			"variety":"Sangiovese",
			"winery":"Casa Raia",
			"avg winter temp":4.2,
			"avg spring temp":15,
			"avg summer temp":27,
			"avg fall temp":16,
			"avg winter sun hour":7,
			"avg spring sun hour":12,
			"avg summer sun hour":16,
			"avg fall sun hour":9,
			"avg daily precip winter":3,
			"avg daily precip spring":3,
			"avg daily precip summer":1,
			"avg daily precip fall":2
	
}
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

{"data": "'data'", "message": "Failed to predict "}


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# Deleting the webservice

service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

